In [1]:
using Distributions, CSV, DataFrames, ProgressMeter
include("../../utils_1d.jl")

MethodOfLines (generic function with 1 method)

In [2]:
T=1.0; r=0.03; R = r; μ=0.03; S0 = 100.0; σ = 0.2; K = 100.0; q = 0.0;

drift(x) = μ*x
diffusion(x) = σ*x
driver(t, x, y, z) = (
        -r*max(y-z/σ, 0.0)
        -R*min(y-z/σ, 0.0)
        -μ*z/σ
        )
# terminal(x) = max(K-x[1],0) 
terminal(x) = max(x-K,0)

bsde = BSDE(T, S0, drift, diffusion, driver, terminal);

In [3]:
Nₜs = [10, 20, 50, 100, 200, 500, 1000]
header = vcat(["measurement_type"], string.(Nₜs))
schemes = [
#     [LawsonEuler(krylov=true, m=100), true],
#     [NorsettEuler(krylov=true, m=100), true],
#     [ETDRK2(krylov=true, m=100), true],
#     [ETDRK3(krylov=true, m=100), true],
#     [ETDRK4(krylov=true, m=100), true],
    [HochOst4(krylov=true, m=100), true]
    # [DP5(), false],
    # [RadauIIA5(), false]
]

### designing grids
domain = [0.0, 2*bsde.X0];
Nₗ = 1000; Δₗ = (bsde.X0-domain[1])/Nₗ;
Nᵣ = 1000; Δᵣ = (domain[2]-bsde.X0)/Nᵣ;

g=50.0
spatial_grid = TavellaRandallGrid(g, g, domain[1], bsde.X0, domain[2], Nₗ, Nᵣ)

TavellaRandallGrid([0.0, 0.16129920473716197, 0.3223903372606429, 0.48327373329789225, 0.6439497281433404, 0.8044186566592799, 0.964680853276306, 1.1247366519943256, 1.2845863863829976, 1.4442303895825717  …  198.55576961041743, 198.715413613617, 198.87526334800566, 199.03531914672368, 199.19558134334073, 199.35605027185665, 199.5167262667021, 199.67760966273937, 199.83870079526284, 200.0], 2001, 1000, 1000)

In [4]:
### BENCHMARKs
function price(t::Float64, s::Float64)
    s = abs(s)
    expiry=T-t
    if expiry >0
        d1 = (log(s / K) + (r-q+(σ^2)/2)*expiry)/(σ*sqrt(expiry))
        d2 = d1-σ*sqrt(expiry)
        call = s*exp(-q*expiry)*cdf(Normal(), d1)-K*exp(-r*expiry)*cdf(Normal(), d2)
    else
        call = max(s-K, 0.0)
    end
    return call
end

price (generic function with 1 method)

In [5]:
df = DataFrame([[],[],[],[],[],[],[], []], header)
attr = schemes[end]
scheme = attr[1]
EXPINT = attr[2]

abs100 = zeros(Float64, length(Nₜs));
abssup = zeros(Float64, length(Nₜs)); 
runtimes = zeros(Float64, length(Nₜs))

@showprogress for (ind, Nₜ) in enumerate(Nₜs)
    print(string("Nₜ=", Nₜ,":\n")); flush(stdout)
    exc_start = time()
    res = MethodOfLines(bsde, spatial_grid, Nₜ, scheme, EXPINT)
    exc_stop = time()
    sol = res[1]; s_grid = res[2]; 

    bs_sol = zeros(spatial_grid.N, Nₜ+1)
    for (index, t) in enumerate((bsde.T/Nₜ).*(0:Nₜ))
        bs_sol[:, index] = price.(Ref(T-t), s_grid);
    end
    abs_err = abs.(bs_sol-sol)
    ind_slice = findall(attr->(attr<1.2*bsde.X0)&&(attr>0.8*bsde.X0), spatial_grid.grid)
    indmin = minimum(ind_slice); indmax = maximum(ind_slice);
    abssup[ind] = maximum(abs_err[indmin:indmax,:])
    abs100[ind] = abs_err[spatial_grid.Nₗ + 1,end]
    runtimes[ind] = exc_stop - exc_start
    print(string("Sup Error=", abssup[ind],":\n")); flush(stdout)
    print(string("Abs Error=", abs100[ind],":\n")); flush(stdout)
    print(string("Runtime[s]=", runtimes[ind],":\n")); flush(stdout)
end
row_sup = vcat(["Sup Error"], string.(abssup)); push!(df,row_sup);
row_100 = vcat(["Abs Error"], string.(abs100)); push!(df,row_100);
row_run = vcat(["Runtime[s]"], string.(runtimes)); push!(df,row_run);

df |> CSV.write(string("result20231112.csv"))

Nₜ=10:
Sup Error=0.5102182089232059:
Abs Error=0.261087368393051:
Runtime[s]=7.163421869277954:
Nₜ=20:
Sup Error=0.13738525483463704:
Abs Error=0.026381907922853998:
Runtime[s]=2.263848066329956:


Progress:  29%|███████████▊                             |  ETA: 0:00:26

Nₜ=50:
Sup Error=0.013835301817113788:
Abs Error=0.0010920765561923673:
Runtime[s]=5.638978004455566:


Progress:  43%|█████████████████▋                       |  ETA: 0:00:22

Nₜ=100:
Sup Error=0.013263038971601304:
Abs Error=0.0004692956087168909:
Runtime[s]=16.602717876434326:


...

Nₜ=500:
Sup Error=0.01325561304776457:
Abs Error=0.0004611782221140004:
Runtime[s]=69.78826093673706:


Progress:  86%|███████████████████████████████████▏     |  ETA: 0:00:22

Nₜ=1000:
Sup Error=0.013255613048041681:
Abs Error=0.0004611782220305116:
Runtime[s]=102.15225195884705:


Progress: 100%|█████████████████████████████████████████| Time: 0:03:53


"result20231112.csv"